# 1. Value Iteration Optimization
Given a maze, find value function of all valid states to terminal states

## Define state space, action space, state transition probability function, two different reward transition function and discount rate

For the following below, we are formulating for a general maze. with maze_grid given as input(defined by dictionary of position to (space, block, goal) )  

1. **State Space**: $\mathcal{S}$ = { (i,j) | maze_grid[(i,j)] == "SPACE" or maze_grid[(i,j)] == "GOAL"}
2. Action Space, We first define the following set. U = {"up", "down", "left", "right"} action_map = {"left":(0,-1),"right":(0,1),"up":(-1,0),"down":(1,0)}. Note that maze_grid is a given set in grid_maze.py. 

**Action Space** will be defined as: $\mathcal{A}(s)$ = {a| $a\in U$,  maze_grid[s+action_map[a]] == "SPACE" or maze_grid[s+action_map[a]] == "GOAL"}


3. **State Transition Probability Function**: $\mathcal{P}(s,a,s')$. Given s and s' are valid state (i.e. mazegrid[s] == "SPACE" and mazegrid[s'] == "SPACE")
\begin{equation}
\mathcal{P}((i,j),"up",(i,j-1)) = 1 \\
\mathcal{P}((i,j),"down",(i,j+1)) = 1\\
\mathcal{P}((i,j),"left",(i-1,j)) = 1\\
\mathcal{P}((i,j),"right",(i+1,j)) = 1 \\
\text{otherwise, it will be 0}
\end{equation}


4. **Reward transition function 1**: all rewards are 0 except when reaching the end goal. i.e.
\begin{equation}
\mathcal{R}_T(\cdot,\cdot,(7,7)) = 1
\end{equation}
This case we will have discount rate = $\gamma <1$

5. **Reward transition function 2**: each move we take, we waste energy and take a cost of -0.01. End goal will still have reward of 1. 
\begin{equation}
\mathcal{R}_T(s,\cdot,s') = -0.01 \text{ if $s'\neq (7,7)$}\\
\mathcal{R}_T(\cdot,\cdot,(7,7)) = 1
\end{equation}
This case we can have discount rate = $\gamma=1$

In [1]:
from grid_maze import *
import sys
sys.path.append("..")

from collections import defaultdict
from rl.markov_decision_process import  StateActionMapping, FinitePolicy, FiniteMarkovDecisionProcess
from rl.markov_process import FiniteMarkovRewardProcess
from rl.markov_process import StateReward
from rl.distribution import Choose,Constant,Categorical
from dataclasses import dataclass
import numpy as np
import matplotlib.pyplot as plt
from rl.dynamic_programming import policy_iteration_result, value_iteration_result
from typing import (Text, Dict, Iterable, Generic, Sequence, Tuple,
                    Mapping, Optional, TypeVar, Generator)

## Formulate the MDP, solve optimal value, optimal policy and find the number of iterations needed to converge

edited code in rl/iterate, line 43 - 65
```
def converge(values: Iterator[X], done: Callable[[X, X], bool]) -> Iterator[X]:
    '''Read from an iterator until two consecutive values satisfy the
    given done function or the input iterator ends.

    Raises an error if the input iterator is empty.

    Will loop forever if the input iterator doesn't end *or* converge.

    '''
    a = next(values, None)
    if a is None:
        return
    i = 1
    yield a

    for b in values:
        if done(a, b):
            print("iterations made = ",i)
            return

        a = b
        i +=1
        yield b
```

In [2]:
def tuple_sum(a,b):
    return (a[0]+b[0],a[1]+b[1])

In [3]:
def getMappingOne(maze_grid) -> Dict[Tuple[int, int], Dict[Text, Categorical[Tuple[int, int]]]]:
    N: Sequence[Tuple[int, int]] = []
    T: Sequence[Tuple[int, int]] = []
    B: Sequence[Tuple[int, int]] = []
    for k,v in maze_grid.items():
        if v == "SPACE":
            N.append(k)
        elif v == "BLOCK":
            B.append(k)
        else:
            T.append(k)
    S: Sequence[Tuple[int, int]] = N+T
    actions = ["up", "down", "left", "right"]
    a_d = {"left":(0,-1),"right":(0,1),"up":(-1,0),"down":(1,0)}
    mapping: Dict[Tuple[int, int], Dict[Text, Categorical[Tuple[int, int]]]] ={}
    for _ in T:
        mapping[_] = None
    
    for s0 in N:
        mapping[s0]:Dict[Text, Categorical[Tuple[int, int]]]={}
        for a in actions:
            if tuple_sum(s0,a_d[a]) in S:
                if tuple_sum(s0,a_d[a]) in T:
                    d: Dict[Tuple[Tuple[int, int]],float] = {(tuple_sum(s0,a_d[a]),1):1}
                else:  ## s1 is in N
                    d: Dict[Tuple[Tuple[int, int]],float] = {(tuple_sum(s0,a_d[a]),0):1}
                mapping[s0][a]= Categorical(d)

    return mapping

In [22]:
mdp = FiniteMarkovDecisionProcess(getMappingOne(maze_grid))
value_star1, policy_star1 = value_iteration_result(mdp, gamma=0.9)

iterations made =  17


In [5]:
value_star1

{(0, 0): 0.2058911320946491,
 (0, 2): 0.31381059609000017,
 (0, 3): 0.34867844010000015,
 (0, 4): 0.31381059609000017,
 (0, 5): 0.28242953648100017,
 (0, 6): 0.25418658283290013,
 (0, 7): 0.22876792454961012,
 (1, 0): 0.22876792454961012,
 (1, 3): 0.38742048900000015,
 (2, 0): 0.25418658283290013,
 (2, 2): 0.38742048900000015,
 (2, 3): 0.43046721000000016,
 (2, 4): 0.47829690000000014,
 (2, 5): 0.5314410000000002,
 (2, 7): 0.5314410000000002,
 (3, 0): 0.28242953648100017,
 (3, 1): 0.31381059609000017,
 (3, 2): 0.34867844010000015,
 (3, 5): 0.5904900000000002,
 (3, 7): 0.5904900000000002,
 (4, 0): 0.25418658283290013,
 (4, 2): 0.31381059609000017,
 (4, 4): 0.5904900000000002,
 (4, 5): 0.6561000000000001,
 (4, 6): 0.7290000000000001,
 (4, 7): 0.6561000000000001,
 (5, 2): 0.28242953648100017,
 (5, 4): 0.5314410000000002,
 (5, 6): 0.81,
 (6, 0): 0.25418658283290013,
 (6, 4): 0.47829690000000014,
 (6, 6): 0.9,
 (6, 7): 1.0,
 (7, 0): 0.28242953648100017,
 (7, 1): 0.31381059609000017,
 (7, 2)

In [6]:
policy_star1

For State (0, 0):
  Do Action down with Probability 1.000
For State (0, 2):
  Do Action right with Probability 1.000
For State (0, 3):
  Do Action down with Probability 1.000
For State (0, 4):
  Do Action left with Probability 1.000
For State (0, 5):
  Do Action left with Probability 1.000
For State (0, 6):
  Do Action left with Probability 1.000
For State (0, 7):
  Do Action left with Probability 1.000
For State (1, 0):
  Do Action down with Probability 1.000
For State (1, 3):
  Do Action down with Probability 1.000
For State (2, 0):
  Do Action down with Probability 1.000
For State (2, 2):
  Do Action right with Probability 1.000
For State (2, 3):
  Do Action right with Probability 1.000
For State (2, 4):
  Do Action right with Probability 1.000
For State (2, 5):
  Do Action down with Probability 1.000
For State (2, 7):
  Do Action down with Probability 1.000
For State (3, 0):
  Do Action right with Probability 1.000
For State (3, 1):
  Do Action right with Probability 1.000
For Stat

In [30]:
def getMappingTwo(maze_grid)-> Dict[Tuple[int, int], Dict[Text, Categorical[Tuple[int, int]]]]:
    N = []
    T = []
    B = []
    for k,v in maze_grid.items():
        if v == "SPACE":
            N.append(k)
        elif v == "BLOCK":
            B.append(k)
        else:
            T.append(k)
    S = N+T
    actions = ["up", "down", "left", "right"]
    a_d = {"left":(0,-1),"right":(0,1),"up":(-1,0),"down":(1,0)}
    rewards = {i:-0.01 for i in N}
    mapping={}
    for _ in T:
        rewards[_] = 1
        mapping[_] = None
    
    for s0 in N:
        mapping[s0]={}
        for a in actions:
            if tuple_sum(s0,a_d[a]) in S:
                if tuple_sum(s0,a_d[a]) in T:
                    d = {(tuple_sum(s0,a_d[a]),1):1}
                else:  ## s1 is in N
                    d = {(tuple_sum(s0,a_d[a]),-0.01):1}
                mapping[s0][a]= Categorical(d)

    return mapping

In [31]:
mdp = FiniteMarkovDecisionProcess(getMappingTwo(maze_grid))
value_star2, policy_star2 = value_iteration_result(mdp, gamma=1)

iterations made =  17


In [32]:
value_star2

{(0, 0): 0.8499999999999999,
 (0, 2): 0.8899999999999999,
 (0, 3): 0.8999999999999999,
 (0, 4): 0.8899999999999999,
 (0, 5): 0.8799999999999999,
 (0, 6): 0.8699999999999999,
 (0, 7): 0.8599999999999999,
 (1, 0): 0.8599999999999999,
 (1, 3): 0.9099999999999999,
 (2, 0): 0.8699999999999999,
 (2, 2): 0.9099999999999999,
 (2, 3): 0.9199999999999999,
 (2, 4): 0.9299999999999999,
 (2, 5): 0.94,
 (2, 7): 0.94,
 (3, 0): 0.8799999999999999,
 (3, 1): 0.8899999999999999,
 (3, 2): 0.8999999999999999,
 (3, 5): 0.95,
 (3, 7): 0.95,
 (4, 0): 0.8699999999999999,
 (4, 2): 0.8899999999999999,
 (4, 4): 0.95,
 (4, 5): 0.96,
 (4, 6): 0.97,
 (4, 7): 0.96,
 (5, 2): 0.8799999999999999,
 (5, 4): 0.94,
 (5, 6): 0.98,
 (6, 0): 0.8699999999999999,
 (6, 4): 0.9299999999999999,
 (6, 6): 0.99,
 (6, 7): 1.0,
 (7, 0): 0.8799999999999999,
 (7, 1): 0.8899999999999999,
 (7, 2): 0.8999999999999999,
 (7, 3): 0.9099999999999999,
 (7, 4): 0.9199999999999999}

In [35]:
policy_star2

For State (0, 0):
  Do Action down with Probability 1.000
For State (0, 2):
  Do Action right with Probability 1.000
For State (0, 3):
  Do Action down with Probability 1.000
For State (0, 4):
  Do Action left with Probability 1.000
For State (0, 5):
  Do Action left with Probability 1.000
For State (0, 6):
  Do Action left with Probability 1.000
For State (0, 7):
  Do Action left with Probability 1.000
For State (1, 0):
  Do Action down with Probability 1.000
For State (1, 3):
  Do Action down with Probability 1.000
For State (2, 0):
  Do Action down with Probability 1.000
For State (2, 2):
  Do Action right with Probability 1.000
For State (2, 3):
  Do Action right with Probability 1.000
For State (2, 4):
  Do Action right with Probability 1.000
For State (2, 5):
  Do Action down with Probability 1.000
For State (2, 7):
  Do Action down with Probability 1.000
For State (3, 0):
  Do Action right with Probability 1.000
For State (3, 1):
  Do Action right with Probability 1.000
For Stat

## Demonstrate that both policy matches 
Note that both require 17 iterations, the answers are printed above.  
And both policies matches

In [33]:
def policyEqual(policy_star1,policy_star2):
    if policy_star1.states()!=policy_star2.states():
        return False
    for s0 in policy_star1.states():
        if policy_star1.act(s0)!=policy_star2.act(s0):
            return False
    return True

In [34]:
policyEqual(policy_star1,policy_star2)

True

# 2. MRP Value Function Approximation


The bellman equation in vector form is
\begin{equation}
v = \mathcal{R}+\gamma\mathcal{P}\cdot v
\end{equation}

where v and $\mathcal{R}$ is n X 1 vector, $\mathcal{P}$ is a n X n matrix,  
Manipulating the above equation, we will get
\begin{equation}
v=(I_n -\gamma \mathcal{P})^{-1}\cdot \mathcal{R}
\end{equation}

$\Phi \cdot \omega$ should be linear to v and therefore linear to $(I_n -\gamma \mathcal{P})^{-1}\cdot \mathcal{R}$.  
In summary, We need to have the following condition
* m = n

* $(I_n -\gamma \mathcal{P})$ should be invertible

* $v = (I_n -\gamma \mathcal{P})^{-1}\cdot \mathcal{R}$ should be in the column space of $\Phi$

# 3. Career Optimization

## specify states, actions, rewards, state-transition probabilities and discount factor
* States: we will only store an integer, the last gotten hourly wages. {$w | 0\leq w\leq W, w\in\mathbb{Z}$}
* Actions: (l,s) where l is number of hours spend on learning and s is number of hours spend on searching a job  
  Actions: {$(l,s)|0\leq l,s \leq H, l+s\leq H$}
  
* Rewards: rewards will just be an integer, it will be the expected reward at the end of day, computed by w*(H-l-s).  $\mathcal{R}_T(w0,(l,s),w1) = w1*(H-l-s)$

* State-Transition probabilities, we let $x\sim Po(\alpha*l)$, then if $w_1,w_2 \leq W$

\begin{equation}
\mathcal{P}(w0,(l,s),w1) = 
\begin{cases} 
P(x=w1-w0) & \text{if $w1-w0>1$} \\
P(x=0)*\frac{\beta \cdot s}{H}    & \text{if $w1=w0$}\\
P(x=1)+P(x=0) *(1-\frac{\beta \cdot s}{H}) & \text{ if $w1=w0+1$}
\end{cases}
\end{equation}

* Gamma: discounting factor, we let it be 0.95

## Implement MDP in python 

In [15]:
@dataclass(frozen=True)
class actions:
    l: int
    s: int

In [16]:
from scipy.stats import poisson

alpha = 0.08
beta = 0.82
gamma = 0.95
H=10
W=30

def get_action_transition_reward_map()-> Dict[int, Dict[actions, Categorical[Tuple[int,float]]]]:
    d: Dict[int, Dict[actions, Categorical[Tuple[int,float]]]] = {}
    for w in range(W+1):
        d1: Dict[actions, Categorical[Tuple[int, float]]] = {}
        for l in range(H+1):
            for s in range(H+1-l):
                work_hours= H-l-s
                reward = w*(work_hours)
                poisson_dist=poisson(alpha*l)
                
                prob_dict: Dict[Tuple[int, float], float] ={}
                new_job_wage = min(w+1,W)
                new_job_prob = beta*s/H
                for adjustment in range(W-w+1):
                    if adjustment==W-w:
                        probability = 1-poisson_dist.cdf(W-w-1)
                    else:
                        probability = poisson_dist.pmf(adjustment)
                    old_job_wage = w+adjustment
                    if new_job_wage>old_job_wage:
                        prob_dict[(new_job_wage,work_hours*new_job_wage)]=new_job_prob*probability
                        prob_dict[(old_job_wage,work_hours*old_job_wage)]=(1-new_job_prob)*probability
                        #basically if new job better, with probability u accept, 1-p is old pay.
                    else:
                        prob_dict[(old_job_wage,work_hours*old_job_wage)]=probability
                d1[actions(l,s)] = Categorical(prob_dict)
        d[w] = d1
    return d
                ## loop over all poisson from 0 till W-w
                ## find new wage from poisson.
                ## next day state form with the reward


## Solve for optimal value function and optimal policy 

In [17]:
careerMDP = FiniteMarkovDecisionProcess(get_action_transition_reward_map())
value_star1, policy_star1 = value_iteration_result(careerMDP, gamma=gamma)

iterations made =  337


## Plot a graph of the optimal policy or print it. provide an intitive explanation 

In [18]:
print(policy_star1)

For State 0:
  Do Action actions(l=10, s=0) with Probability 1.000
For State 1:
  Do Action actions(l=10, s=0) with Probability 1.000
For State 2:
  Do Action actions(l=10, s=0) with Probability 1.000
For State 3:
  Do Action actions(l=10, s=0) with Probability 1.000
For State 4:
  Do Action actions(l=10, s=0) with Probability 1.000
For State 5:
  Do Action actions(l=10, s=0) with Probability 1.000
For State 6:
  Do Action actions(l=10, s=0) with Probability 1.000
For State 7:
  Do Action actions(l=10, s=0) with Probability 1.000
For State 8:
  Do Action actions(l=10, s=0) with Probability 1.000
For State 9:
  Do Action actions(l=10, s=0) with Probability 1.000
For State 10:
  Do Action actions(l=10, s=0) with Probability 1.000
For State 11:
  Do Action actions(l=10, s=0) with Probability 1.000
For State 12:
  Do Action actions(l=10, s=0) with Probability 1.000
For State 13:
  Do Action actions(l=8, s=0) with Probability 1.000
For State 14:
  Do Action actions(l=5, s=0) with Probabilit

Well, if your salary is low, we should boost our salary by learning more. when your salary becomes higher, it is more worth it to work more than to learn. It is not worth it to search for new job as the pay rise of the new job has a maximum of 1 dollar/hour and there's an uncertainty on whether we are going to get a new job offer. Besides, it's better to spend it on learning more things so we can get a pay raise from the old job.